## Parquet to SensorThings Streamlike

In [1]:
import csv
import re
import math
import time
import random
import numpy as np
import sys
import json
import requests
import os
import Helper

In [2]:
# dirList: sorted list of dir names
dirList = os.listdir(Helper.Helper.DataHome + "outAllSortByTimeStampAndIDBigAll/")
dirList = [ x for x in dirList if "2016" in x ]
dirList = sorted(dirList, key= lambda myDir: int(myDir.split("=")[1]))



In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
#spark conf
conf = ( SparkConf()
         .setMaster("local[*]")
         .setAppName('pyspark')
        )
ss = SparkSession.builder.config(conf=conf).getOrCreate()
sc = ss.sparkContext


In [4]:
#Common things
mytypes = ["Air-Temperature","Air-Humidity","Air-Pressure"]
urlHome = Helper.Helper.Frosts["dev01"] + ':8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
urlSensors = urlHome + '/Sensors'
urlObservedProperty = urlHome + '/ObservedProperties'
urlDataStream = urlHome + '/Datastreams'
urlFoI = urlHome + '/FeaturesOfInterest'

In [5]:
myPrefix = ""

In [6]:
from pyspark.sql import functions as F

from datetime import tzinfo, timedelta, datetime

myDir = dirList[0]
inputDir = Helper.Helper.DataHome + "outAllSortByTimeStampAndIDBigAll/"+myDir
dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
idDF = dataFileDF["ID","Latitude","Longitude"]

urlInst = Helper.Helper.Inst_URL
idDF = idDF.withColumn("LatLon",F.array("Latitude","Longitude"))
idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))

idDF = idDF.withColumn("ThUID",F.format_string("Things/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
idDF = idDF.withColumn("SensorUID",F.format_string("Sensors/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
idDF = idDF.withColumn("DsUID",F.format_string("Datastreams/%s/MS/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))


In [18]:
idDF.rdd.first()

Row(ID=u'IALFELDL8', LatLon=[u'51.982', u'9.847'], ThUID=u'Things/edu.teco.wang/WG-WeatherStation-IALFELDL8', SensorUID=u'Sensors/edu.teco.wang/WG-WeatherStation-IALFELDL8', DsUID=u'Datastreams/edu.teco.wang/WG-WeatherStation-IALFELDL8')

## Register ObservedProperties

In [7]:
#Observed Property
for mytype in mytypes:
    data ={
        "name": "Air-Temperature",
        "description": "The degree or intensity of heat present in the area",
        "definition": "http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature",
        "@iot.id": myPrefix + "ObservedProperty/" + mytype
    }
    p = requests.post(urlObservedProperty, json.dumps(data))
    print(data)
    if (p.status_code  == 201):
        print(201)
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

{'definition': 'http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature', '@iot.id': 'ObservedProperty/Air-Temperature', 'name': 'Air-Temperature', 'description': 'The degree or intensity of heat present in the area'}
201
{'definition': 'http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature', '@iot.id': 'ObservedProperty/Air-Humidity', 'name': 'Air-Temperature', 'description': 'The degree or intensity of heat present in the area'}
201
{'definition': 'http://www.qudt.org/qudt/owl/1.0.0/quantity/Instances.html#AreaTemperature', '@iot.id': 'ObservedProperty/Air-Pressure', 'name': 'Air-Temperature', 'description': 'The degree or intensity of heat present in the area'}
201


In [ ]:
from pyspark.sql import functions as F

from datetime import tzinfo, timedelta, datetime

def registerEntities(row):
    #Thing
    
    data =  {
        "name": "WG-WeatherStation-" + row.ID,
        "description": "This is DWD-Sensor-" + row.ID,
        "@iot.id": myPrefix + row.ThUID,
        "Locations": [
        {
            "name": row.ThUID,
            "description": "This is the location of WG-Sensor-" + row.ID,
            "encodingType": "application/vnd.geo+json",
            "location": {
              "type": "Point",
              "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
            },
            "@iot.id": myPrefix+"Location/" + row.ThUID 
        }
      ]
        }
    p = requests.post(urlThings, json.dumps(data))
    print("Thing:")
    print(data)
    if (p.status_code  == 201):
        print(201)
        
    else:
        print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    #Sensor

    deviceAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row.ID
    for mytype in mytypes:
        data = {
            "name": "WG-WeatherStation-" + row.ID + "-" + "" + mytype,
            "description": "This is a Sensor from Netatmo Weather Station",
            "encodingType": "application/pdf",
            "metadata": deviceAddr,
            "@iot.id": myPrefix+row.SensorUID +"-"+ mytype
        }
        p = requests.post(urlSensors, json.dumps(data))
        print("Sensor:")
        print(data)
        if (p.status_code  == 201):
            print(201)
            
            
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    #DataStream

    for mytype in mytypes:
        data = {
          "name": "MS/LiveMeasure",
          "description": "Datastream for recording temperature",
          "observationType": "http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement",
          "@iot.id":myPrefix + row.DsUID + "-" + mytype + "/MS/LiveMeasure",
          "unitOfMeasurement": {
            "name": "Degree Celsius",
            "symbol": "degC",
            "definition": "http://www.qudt.org/qudt/owl/1.0.0/unit/Instances.html#DegreeCelsius"
          },
          "Thing":{"@iot.id":myPrefix+row.ThUID},
          "ObservedProperty":{"@iot.id":myPrefix+"ObservedProperty/" + mytype},
          "Sensor":{"@iot.id":myPrefix+row.SensorUID +"-"+ mytype}
        }
        p = requests.post(urlDataStream, json.dumps(data))
        print("DS:")
        print(data)
        if (p.status_code  == 201):
            print(201)
            
        else:
            print("Error:", p.status_code)

        for chunk in p.iter_content(chunk_size=128):
            print(chunk)

    # FoI
    data = {
      "name": "Weather Station-" + row.ID,
      "description": "Weather Station-" + row.ID,
      "encodingType": "application/vnd.geo+json",
        "@iot.id": myPrefix +"FoI/" +myPrefix+ row.DsUID + "-Air-Temperature/MS/LiveMeasure/"+row.LatLon[0]+":"+row.LatLon[1],
      "feature": {
        "type": "Point",
        "coordinates": [float(row.LatLon[0]), float(row.LatLon[1])]
      }
    }
    p = requests.post(urlFoI, json.dumps(data))
    print("FOI:")
    print(data)
    if (p.status_code  == 201):
        print(201)
        
        
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)




for myDir in dirList:
    inputDir = Helper.Helper.DataHome + "outAllSortByTimeStampAndIDBigAll/"+myDir
    dataFileDF = ss.read.option("basepath",inputDir).parquet(inputDir)#+"TimeStamp=20160504/ID=I72406BI1")
    dataFileDF = dataFileDF.withColumnRenamed("AitTemperature","AirTemperature")
    dataFileDF = dataFileDF.filter("(not isnull(ID)) and ID !=''")
    idDF = dataFileDF["ID","Latitude","Longitude"]
    
    urlInst = Helper.Helper.Inst_URL
    idDF = idDF.withColumn("LatLon",F.array("Latitude","Longitude"))
    idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))

    idDF = idDF.withColumn("ThUID",F.format_string("Thing/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
    idDF = idDF.withColumn("SensorUID",F.format_string("Sensor/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))
    idDF = idDF.withColumn("DsUID",F.format_string("Datastream/%s/WG-WeatherStation-%s",F.lit(urlInst),idDF.ID))


    for hourInt in range(0,24):
        oneHourDataFileDF = dataFileDF.rdd.filter(lambda row: int(row.Time.split(':')[0]) == hourInt)
        print(oneHourDataFileDF.count())
        for row in oneHourDataFileDF.collect():
            

            class TZ(tzinfo):
                def utcoffset(self, dt): return timedelta(minutes=120)
            isoTime =datetime(int(row.Date.split('-')[0]),
                         int(row.Date.split("-")[1]),
                         int(row.Date.split("-")[2]),
                         int(row.Time.split(':')[0]),
                         int(row.Time.split(':')[1]),
                         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()




            urlObs = urlHome+"/Datastreams('"+myPrefix+"Datastream/" + urlInst + "/WG-WeatherStation-" +row.ID+ "-" + "Air-Temperature/MS/LiveMeasure')/Observations"
            print(urlObs)
            data = {
              "phenomenonTime": isoTime,
              "resultTime" : isoTime,
              "result" : row.AirTemperature,
              "FeatureOfInterest": {
                  "@iot.id": myPrefix +"FoI/" + myPrefix + "Datastream/"+urlInst +"/WG-WeatherStation-"+row.ID + "-Air-Temperature/MS/LiveMeasure/" +row.Latitude+":"+row.Longitude
              }
            }
            p = requests.post(urlObs, json.dumps(data))

            if (p.status_code  == 201):
                print("+1")
                print(p.headers["location"])
            else:
                print("Error:", p.status_code)
                #print(data)
                #print(idDF.rdd.take(5))
                registerEntities(idDF.rdd.filter(lambda aRow:aRow.ID == row.ID).first())

            for chunk in p.iter_content(chunk_size=128):
                print(chunk)
    time.sleep(10)
    

14761
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-ILDINGHA4-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9bb080d8-67e0-11e8-8fe7-c334c896f57e')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-ILDINGHA4-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9bb4fbfe-67e0-11e8-8fe7-9f5ff868473c')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-ILDINGHA4-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9bb9e272-67e0-11e8-8fe7-2799111bc85d')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-ILDINGHA4-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Ser

+1
http://localhost:8080/FROST-Server/v1.0/Observations('9c4dfa7a-67e0-11e8-8fe7-f3a63a0982b4')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IAICHTAL2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9c50fe64-67e0-11e8-8fe7-57adbc664148')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IAICHTAL2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9c544d4e-67e0-11e8-8fe7-d31509bb8af1')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IAICHTAL2-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9c572078-67e0-11e8-8fe7-dbc55e4cd5c8')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-

+1
http://localhost:8080/FROST-Server/v1.0/Observations('9cd3bc8c-67e0-11e8-8fe7-e7479ae5daaa')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IMUNICH10-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9cd6a712-67e0-11e8-8fe7-f7345b72b438')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IHERBORN7-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9cda25b8-67e0-11e8-8fe7-c7a8b1ae495a')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IHERBORN7-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9cdddaf0-67e0-11e8-8fe7-7f7e9586692c')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-

+1
http://localhost:8080/FROST-Server/v1.0/Observations('9d53c2b0-67e0-11e8-8fe7-9fa1951c5c75')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IWIESBAD233-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9d5a288a-67e0-11e8-8fe7-f75ac8bdf8e8')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IWIESBAD233-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9d5d8908-67e0-11e8-8fe7-9f1b792f7db1')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IWIESBAD233-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9d610236-67e0-11e8-8fe7-d3cbbd39c7dc')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherSt

+1
http://localhost:8080/FROST-Server/v1.0/Observations('9dface48-67e0-11e8-8fe7-cf3c9b90b0b7')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IWEINHEI19-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9dfdc698-67e0-11e8-8fe7-dbe164a23e7e')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IWEINHEI19-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9e00fdb8-67e0-11e8-8fe7-e784f22eb1ce')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStation-IWEINHEI19-Air-Temperature/MS/LiveMeasure')/Observations
+1
http://localhost:8080/FROST-Server/v1.0/Observations('9e03a716-67e0-11e8-8fe7-0baa871b964d')
http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0/Datastreams('Datastream/edu.teco.wang/WG-WeatherStati

...

Thing:
{'@iot.id': u'Thing/edu.teco.wang/WG-WeatherStation-INORDRHE284', 'Locations': [{'location': {'type': 'Point', 'coordinates': [51.929, 8.845]}, '@iot.id': u'Location/Thing/edu.teco.wang/WG-WeatherStation-INORDRHE284', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-INORDRHE284', 'description': u'This is the location of WG-Sensor-INORDRHE284'}], 'name': u'WG-WeatherStation-INORDRHE284', 'description': u'This is DWD-Sensor-INORDRHE284'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=INORDRHE284', 'encodingType': 'application/pdf', '@iot.id': u'Sensor/edu.teco.wang/WG-WeatherStation-INORDRHE284-Air-Temperature', 'name': u'WG-WeatherStation-INORDRHE284-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=I

Thing:
{'@iot.id': u'Thing/edu.teco.wang/WG-WeatherStation-IBERGKAM5', 'Locations': [{'location': {'type': 'Point', 'coordinates': [51.646, 7.661]}, '@iot.id': u'Location/Thing/edu.teco.wang/WG-WeatherStation-IBERGKAM5', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IBERGKAM5', 'description': u'This is the location of WG-Sensor-IBERGKAM5'}], 'name': u'WG-WeatherStation-IBERGKAM5', 'description': u'This is DWD-Sensor-IBERGKAM5'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IBERGKAM5', 'encodingType': 'application/pdf', '@iot.id': u'Sensor/edu.teco.wang/WG-WeatherStation-IBERGKAM5-Air-Temperature', 'name': u'WG-WeatherStation-IBERGKAM5-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IBERGKAM5', 'encodi

Thing:
{'@iot.id': u'Thing/edu.teco.wang/WG-WeatherStation-IVOGT7', 'Locations': [{'location': {'type': 'Point', 'coordinates': [47.772, 9.766]}, '@iot.id': u'Location/Thing/edu.teco.wang/WG-WeatherStation-IVOGT7', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IVOGT7', 'description': u'This is the location of WG-Sensor-IVOGT7'}], 'name': u'WG-WeatherStation-IVOGT7', 'description': u'This is DWD-Sensor-IVOGT7'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IVOGT7', 'encodingType': 'application/pdf', '@iot.id': u'Sensor/edu.teco.wang/WG-WeatherStation-IVOGT7-Air-Temperature', 'name': u'WG-WeatherStation-IVOGT7-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IVOGT7', 'encodingType': 'application/pdf', '@

Thing:
{'@iot.id': u'Thing/edu.teco.wang/WG-WeatherStation-ISANKTBL2', 'Locations': [{'location': {'type': 'Point', 'coordinates': [47.763, 8.126]}, '@iot.id': u'Location/Thing/edu.teco.wang/WG-WeatherStation-ISANKTBL2', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-ISANKTBL2', 'description': u'This is the location of WG-Sensor-ISANKTBL2'}], 'name': u'WG-WeatherStation-ISANKTBL2', 'description': u'This is DWD-Sensor-ISANKTBL2'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=ISANKTBL2', 'encodingType': 'application/pdf', '@iot.id': u'Sensor/edu.teco.wang/WG-WeatherStation-ISANKTBL2-Air-Temperature', 'name': u'WG-WeatherStation-ISANKTBL2-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=ISANKTBL2', 'encodi

Thing:
{'@iot.id': u'Thing/edu.teco.wang/WG-WeatherStation-IDSSELDO42', 'Locations': [{'location': {'type': 'Point', 'coordinates': [51.189, 6.789]}, '@iot.id': u'Location/Thing/edu.teco.wang/WG-WeatherStation-IDSSELDO42', 'encodingType': 'application/vnd.geo+json', 'name': u'Thing/edu.teco.wang/WG-WeatherStation-IDSSELDO42', 'description': u'This is the location of WG-Sensor-IDSSELDO42'}], 'name': u'WG-WeatherStation-IDSSELDO42', 'description': u'This is DWD-Sensor-IDSSELDO42'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IDSSELDO42', 'encodingType': 'application/pdf', '@iot.id': u'Sensor/edu.teco.wang/WG-WeatherStation-IDSSELDO42-Air-Temperature', 'name': u'WG-WeatherStation-IDSSELDO42-Air-Temperature', 'description': 'This is a Sensor from Netatmo Weather Station'}
('Error:', 500)
Failed to store data.
Sensor:
{'metadata': u'https://www.wunderground.com/personal-weather-station/dashboard?ID=IDSSELDO42

In [ ]:
Datastream/edu.teco.wang/WG-WeatherStation-IBAESWEI4-Air-Temperature/MS/LiveMeasure
Datastream/edu.teco.wang/WG-WeatherStation-IBAESWEI4-Air-Temperature/MS/LiveMeasure

/FoI/Datastream/edu.teco.wang/WG-WeatherStation-IBIRKENH15-Air-Temperature/MS/LiveMeasure/49.480:8.265

In [ ]:
urlObs = urlHome+"/Datastreams('"+myPrefix+"edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"

p = requests.post(urlObs, json.dumps(
{
  "phenomenonTime": isoTime,
  "resultTime" : isoTime,
  "result" : row.AirTemperature,
  "FeatureOfInterest": {
      "@iot.id": myPrefix + urlInst+"/"+row.ID + "/FoI/"+row.ID+"/"+row.Latitude+":"+row.Longitude
  }
}
))

if (p.status_code  == 201):
    print(p.headers["location"])
else:
    print("Error:", p.status_code)

for chunk in p.iter_content(chunk_size=128):
    print(chunk)

In [ ]:

#idDF = idDF.withColumn("FoIUID",F.format_string("%s/FeaturesOfInterest/%s",idDF.ThID,idDF.ID))
#idDF = idDF.withColumn("SensorUID",F.format_string("%s/Sensors/%s",idDF.ThID,idDF.ID))


In [ ]:
row = idDF.collect()[0]

row

In [ ]:
# Observations
dataFileDF.show(10)

In [ ]:
row = dataFileDF.collect()[0]
row

In [ ]:
from datetime import tzinfo, timedelta, datetime
for row in dataFileDF.collect():
    

    class TZ(tzinfo):
        def utcoffset(self, dt): return timedelta(minutes=120)
    isoTime =datetime(int(row.Date.split('-')[0]),
                 int(row.Date.split("-")[1]),
                 int(row.Date.split("-")[2]),
                 int(row.Time.split(':')[0]),
                 int(row.Time.split(':')[1]),
                 int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()
    
    
    
    
    urlObs = urlHome+"/Datastreams('edu.teco.wang/"+row.ID+"/Datastreams/" + "Air-Temperature" + "-" + row.ID+"')/Observations"
    data ={
      "phenomenonTime": isoTime,
      "resultTime" : isoTime,
      "result" : row.AirTemperature
    }
    p = requests.post(urlObs, json.dumps(data))
    if (p.status_code  == 201):
        print(p.headers["location"])
    else:
        print("Error:", p.status_code)

    for chunk in p.iter_content(chunk_size=128):
        print(chunk)


In [ ]:
urlObs

In [ ]:
p.headers["location"]

In [ ]:
from datetime import tzinfo, timedelta, datetime
row = dataFileDF.first()
row.Date.split("-")[0]
class TZ(tzinfo):
    def utcoffset(self, dt): return timedelta(minutes=120)
datetime(int(row.Date.split('-')[0]),
         int(row.Date.split("-")[1]),
         int(row.Date.split("-")[2]),
         int(row.Time.split(':')[0]),
         int(row.Time.split(':')[1]),
         int(row.Time.split(':')[2]),tzinfo=TZ()).isoformat()

## Tryouts

In [ ]:
row = idDF.collect()[1]
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
data =  {
        "name": "DWD-Sensor-noSens" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/noSensblblb" + row[0]
    }
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print(201)
else:
    print("Error:", p.status_code)
    
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:

idDF = idDF.groupBy(idDF.ID).agg(F.first(idDF.LatLon).alias("LatLon"))#.show(3)#.agg(F.first(idDF.Longitude))

In [ ]:
urlHome = 'http://smartaqnet-dev01.teco.edu:8080/FROST-Server/v1.0'
#urlHome = 'http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'

In [ ]:

#idDF2 = idDF.select("ID","LatLon").rdd.map(DfToSensorthings)

In [ ]:
row = idDF.collect()[1]
import requests
urlHome = 'http://smartaqnet.teco.edu:8080/FROST-Server/v1.0'
urlThings = urlHome + '/Things'
sensorAddr = "https://www.wunderground.com/personal-weather-station/dashboard?ID="+row[0]
data =  {
        "name": "DWD-Sensor-" + row[0],
        "description": "DWD_Sensor-" + row[0],
        "@iot.id": "DWD-Sensor/" + row[0]
    }
p = requests.get(urlThings+"('DWD-Sensor/noSensblbl" + row[0]+"')")
if (p.status_code  == 200):
    print(200)
else:
    print("Error:", p.status_code)
    
for chunk in p.iter_content(chunk_size=128):
    print(chunk)

In [ ]:
row

In [ ]:
p = requests.post(urlThings, json.dumps(data))
if (p.status_code  == 201):
    print("Creation successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)
        

In [ ]:
#Delete a thing

deleteID = urlThings + "('DWD-Sensors/IBREUNA2')"
print(deleteID)
p = requests.delete(urlThings)
if (p.status_code  == 201):
    print("Deletion successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:
#Query a thing

getID = urlThings + "(\"teco.edu/Test-2\")"
print(getID)
p = requests.get(urlThings)
if (p.status_code  == 201):
    print("Get successful")
else:
    print("Error:", p.status_code)
    for chunk in p.iter_content(chunk_size=128):
        print(chunk)

In [ ]:
p.content

In [ ]:
sensors = idDF.collect()
for sensor in sensors:
    print(DfToSensorthingsPost(sensor))
    #self link

In [ ]:
import requests

In [ ]:
dataFileDF = ss.read.parquet(inputDir)
outDir = "./outDir/outAllAll5/"
print(csv.__file__)

In [ ]:
validFileNames = [inputDir+f for f in os.listdir(inputDir) if ('.' not in f) and ("part-" in f)]
validFileNames

In [ ]:
for aFile in validFileNames:
    with open(aFile) as csvfile:
        for row in csvfile:
            k = row[1:9]
            v = row[13:-3]+'\n'
            vNew = re.sub(';','\n',v)
            outFile = open(outDir+k,"w")
            outFile.write(vNew)
            outFile.close()


In [ ]:
row = csvfile.readline()

In [ ]:
k = row[1:9]
v = row[13:-3].split(";")
k,v

In [ ]:
with open(inputFile) as csvfile:
    for row in csvfile:
        k = row[1:9]
        v = row[13:-3]+'\n'
        vNew = re.sub(';','\n',v)
        outFile = open(outDir+k,"w")
        outFile.write(vNew)
        outFile.close()

In [ ]:
import re

print(vNew)

In [ ]:
outFile = open(outDir+k,"w")
outFile.write(vNew)
outFile.close()

In [ ]:
with open(inputFile,'rb') as csvfile:
    reader = csv.Reader(csvfile)
    row = reader.__next__()
row

In [ ]:
row

In [ ]:
all=string.maketrans('','')
nodigs=all.translate(all, string.digits)

In [ ]:
class Del:
  def __init__(self, keep=string.digits):
    self.comp = dict((ord(c),c) for c in keep)
  def __getitem__(self, k):
    return self.comp.get(k)

In [ ]:
with open(inputPath) as csvfile:
    reader = csv.DictReader(csvfile)
    for i in range(10):
        row = reader.__next__()
        
        lineID = row['id']
        timeStamp = row['time'].translate(Del())
        print(lineID, timeStamp)
        